In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
!pip install rouge

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from nltk.tokenize import sent_tokenize
import numpy as np
import networkx as nx
from rouge import Rouge
from datasets import load_dataset

# Function to read and tokenize text
def read_article(text):
    sentences = sent_tokenize(text)
    return [sentence.replace("[^a-zA-Z0-9]", " ") for sentence in sentences]

# Calculate cosine similarity between two sentences
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []

    sent1 = [w.lower() for w in sent1 if w not in stopwords]
    sent2 = [w.lower() for w in sent2 if w not in stopwords]

    all_words = list(set(sent1 + sent2))
    vector1 = [sent1.count(w) for w in all_words]
    vector2 = [sent2.count(w) for w in all_words]

    return 1 - cosine_distance(vector1, vector2)

# Build similarity matrix for sentences
def build_similarity_matrix(sentences, stop_words):
    matrix = np.zeros((len(sentences), len(sentences)))

    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                matrix[i][j] = sentence_similarity(sentences[i], sentences[j], stop_words)

    return matrix

# Generate summary based on top-ranked sentences
def generate_summary(text, top_n):
    stop_words = stopwords.words('english')
    sentences = read_article(text)
    similarity_matrix = build_similarity_matrix(sentences, stop_words)
    sentence_graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(sentence_graph)

    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    summary = " ".join([ranked_sentences[i][1] for i in range(min(top_n, len(ranked_sentences)))])
    return summary, len(sentences)

# Evaluate summaries using ROUGE
def evaluate_summary(generated_summary, reference_summary):
    rouge = Rouge()
    scores = rouge.get_scores(generated_summary, reference_summary)
    return scores

In [ ]:
dataset = load_dataset("percins/IN-ABS")
articles = dataset['train']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/726 [00:00<?, ?B/s]

(…)-00000-of-00001-807d038895b1783e.parquet:   0%|          | 0.00/78.0M [00:00<?, ?B/s]

(…)-00000-of-00001-410ed27cbab79f30.parquet:   0%|          | 0.00/11.0M [00:00<?, ?B/s]

(…)-00000-of-00001-5eb2cf2771ef7231.parquet:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5346 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/712 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1070 [00:00<?, ? examples/s]

In [ ]:
print(len(articles))

5346


In [ ]:
rouge_1_r = rouge_1_p = rouge_1_f = 0
rouge_2_r = rouge_2_p = rouge_2_f = 0
rouge_l_r = rouge_l_p = rouge_l_f = 0

In [ ]:
for idx, article in enumerate(articles, start=1):
    text = article['text']
    ground_truth = article['summary']
    summary, _ = generate_summary(text, top_n=5)
    scores = evaluate_summary(summary, ground_truth)

    rouge_1_r += scores[0]['rouge-1']['r']
    rouge_1_p += scores[0]['rouge-1']['p']
    rouge_1_f += scores[0]['rouge-1']['f']

    rouge_2_r += scores[0]['rouge-2']['r']
    rouge_2_p += scores[0]['rouge-2']['p']
    rouge_2_f += scores[0]['rouge-2']['f']

    rouge_l_r += scores[0]['rouge-l']['r']
    rouge_l_p += scores[0]['rouge-l']['p']
    rouge_l_f += scores[0]['rouge-l']['f']

    if idx % 100 == 0:
        print(f"{idx} rows complete")

100 rows complete


NameError: name 'i' is not defined

In [ ]:
num_articles = idx
print(num_articles)

87


In [ ]:
print("Average ROUGE-1:", {'r': rouge_1_r / num_articles, 'p': rouge_1_p / num_articles, 'f': rouge_1_f / num_articles})
print("Average ROUGE-2:", {'r': rouge_2_r / num_articles, 'p': rouge_2_p / num_articles, 'f': rouge_2_f / num_articles})
print("Average ROUGE-L:", {'r': rouge_l_r / num_articles, 'p': rouge_l_p / num_articles, 'f': rouge_l_f / num_articles})


Average ROUGE-1: {'r': 0.2882911711616009, 'p': 0.5523277580582466, 'f': 0.3626875915548062}
Average ROUGE-2: {'r': 0.1378114271242719, 'p': 0.3121851527086128, 'f': 0.18045894168612514}
Average ROUGE-L: {'r': 0.257005574833642, 'p': 0.4961558809506975, 'f': 0.32439130922717824}
